In [8]:
import numpy as np
import matplotlib.pyplot as plt
import imageio
import time
import math

from scipy.io.wavfile import read

In [9]:
def diffProcessor(dataX, dataY, dataZ, song='default', side='XD'):
    dataX_dt = np.ones(dataX.shape)
    dataY_dt = np.ones(dataY.shape)
    dataZ_dt = np.ones(dataZ.shape)
    dt_plot = np.ones((len(dataX), 3))
    for ia in range(1,len(dataX)):
        dataX_dt[ia] = dataX[ia] - dataX[ia - 1]
        dataY_dt[ia] = dataY[ia] - dataY[ia - 1]
        dataZ_dt[ia] = dataZ[ia] - dataZ[ia - 1]
        
        dt_plot[ia,0] = np.sum(dataX_dt[ia])
        dt_plot[ia,1] = np.sum(dataY_dt[ia])
        dt_plot[ia,2] = np.sum(dataZ_dt[ia])
    
    # OPTIONAL
    debugName = '' # set to blank normally.
    
    np.savetxt(f'./TriCam/{song.upper()}/Cleaned/{song}_cut_{side}_dtX.csv', dataX_dt, delimiter=',', fmt='%d')
    np.savetxt(f'./TriCam/{song.upper()}/Cleaned/{song}_cut_{side}{debugName}_dtY.csv', dataY_dt, delimiter=',', fmt='%d')
    np.savetxt(f'./TriCam/{song.upper()}/Cleaned/{song}_cut_{side}{debugName}_dtZ.csv', dataZ_dt, delimiter=',', fmt='%d')
    
    np.savetxt(f'./TriCam/{song.upper()}/Cleaned/{song}_cut_{side}{debugName}_combo_X.csv', dataX, delimiter=',', fmt='%d')
    np.savetxt(f'./TriCam/{song.upper()}/Cleaned/{song}_cut_{side}{debugName}_combo_Y.csv', dataY, delimiter=',', fmt='%d')
    np.savetxt(f'./TriCam/{song.upper()}/Cleaned/{song}_cut_{side}{debugName}_combo_Z.csv', dataZ, delimiter=',', fmt='%d')
    print('Processed Differentials')

In [10]:
def handSwapper(data_L, data_R):
    if data_L.shape != data_R.shape:
        print('Invalid data')
    else:
        swap_count = 0
        dif_L = np.ones(len(data_L))
        dif_R = dif_L
        dif_LS = dif_L
        dif_RS = dif_L
        thresh = 100
        for ia in range(1,len(data_L)):
            dif_L = data_L[ia] - data_L[ia - 1]
            dif_R = data_R[ia] - data_R[ia - 1]
            dif_LS = data_L[ia] - data_R[ia - 1]
            dif_RS = data_R[ia] - data_L[ia - 1]
#             print(ia, np.sum(dif_L), np.sum(dif_R), np.sum(dif_LS), np.sum(dif_RS))
            if (abs(np.sum(dif_L))+abs(np.sum(dif_R))) < (abs(np.sum(dif_LS))+abs(np.sum(dif_RS))):
                swap_count += 1
                temp = data_L[ia]
                data_L[ia] = data_R[ia]
                data_R[ia] = temp
        print(f'Swapped {swap_count} times.')


In [11]:
def gapCleaner(data_clean):
    gap_start = np.zeros(len(data_clean[0]))
    gap_end = gap_start
    gap_len = 0
    gap_found = 0
    gap_fill = 0
    gap = False
    for ia in range(len(data_clean)):
        if np.count_nonzero(data_clean[ia]) == 0:
            gap = True
            gap_len += 1
            gap_found += 1
        elif gap:
            if gap_len == 1:
                data_clean[ia - 1] = (data_clean[ia] + data_clean[ia - 2]) / 2
            else:
                avg = (data_clean[ia] - data_clean[ia - gap_len - 1])/(np.ones(len(data_clean[0]))*(gap_len+2))
                for ib in range(1,gap_len+1):
                    gap_fill += 1
                    data_clean[ia - ib] = data_clean[ia - ib + 1] - (avg)
            gap_len = 0
            gap = False
        else:
            gap = False
            gap_len = 0
    print(f'Found {gap_found} gaps, filled in {gap_fill}.')
    #return data_clean

In [12]:
def xyRotate45(dataX, dataY, direction):
    x_rot = np.ones(len(dataX[0]))
    y_rot = x_rot
    for ia in range(len(dataX)):
        x_rot = dataX[ia] * (np.ones(len(dataX[0]))*math.cos(direction*math.pi/4)) + dataY[ia] * (np.ones(len(dataX[0]))*(-1)*math.sin(direction*math.pi/4))
        y_rot = dataX[ia] * (np.ones(len(dataX[0]))*math.sin(direction*math.pi/4)) + dataY[ia] * (np.ones(len(dataX[0]))*math.cos(direction*math.pi/4))
        dataX[ia] = x_rot
        dataY[ia] = y_rot
        

In [13]:
def main(song, side):
    fileLoc = f'./TriCam/{song.upper()}/{song}_{side}'

    dataX1 = np.loadtxt(f'{fileLoc}_RX.csv', delimiter=",", dtype=int)
    dataY1 = np.loadtxt(f'{fileLoc}_RY.csv', delimiter=",", dtype=int)
    dataZ1 = np.loadtxt(f'{fileLoc}_RZ.csv', delimiter=",", dtype=int)
    
    
    dataX = np.ones((len(dataX1), 42))
    dataY = np.ones((len(dataX1), 42))
    dataZ = np.ones((len(dataX1), 42))
    
    dataX2 = ((np.loadtxt(f'{fileLoc}_LX.csv', delimiter=",", dtype=int)))
    dataY2 = ((np.loadtxt(f'{fileLoc}_LY.csv', delimiter=",", dtype=int)))
    dataZ2 = ((np.loadtxt(f'{fileLoc}_LZ.csv', delimiter=",", dtype=int)))
    
#     handSwapper(dataX1,dataX2)
#     handSwapper(dataY1,dataY2)
#     handSwapper(dataZ1,dataZ2)
    
    gapCleaner(dataX1)
    gapCleaner(dataY1)
    gapCleaner(dataZ1)
    
    gapCleaner(dataX2)
    gapCleaner(dataY2)
    gapCleaner(dataZ2)
    
    for ia in range(len(dataX)):
        dataX[ia] = np.concatenate((dataX2[ia],dataX1[ia]))
        dataY[ia] = np.concatenate((dataY2[ia],dataY1[ia]))
        dataZ[ia] = np.concatenate((dataZ2[ia],dataZ1[ia]))
    
    # Processors
    if side == 'left':
        xyRotate45(dataX, dataZ, -1)
    elif side == 'right':
        xyRotate45(dataX, dataZ, 1)
    
    diffProcessor(dataX, dataY, dataZ, song, side)

In [14]:
if __name__ == "__main__":
    songs = ['wu2s','drwn','strfl']
    sides = ['left','right','center']
    time_top_start = time.time()
    for song in songs:
        for side in sides:
            top_proc_start = time.time()
            main(song, side)
            proc_time = time.time() - top_proc_start
            print(f'Processed {song}_{side}. Took {int(proc_time/60)}:{int(proc_time%60)}')
    time_top_start = time_top_start - time.time()
    print(f'Processed {song}_{side}. Took {int(time_top_start/60)}:{int(time_top_start%60)}')
    print("Done XD")

Found 3157 gaps, filled in 2961.
Found 3157 gaps, filled in 2961.
Found 3157 gaps, filled in 2961.
Found 433 gaps, filled in 277.
Found 433 gaps, filled in 277.
Found 433 gaps, filled in 277.
Processed Differentials
Processed wu2s_left. Took 0:1
Found 848 gaps, filled in 604.
Found 848 gaps, filled in 604.
Found 848 gaps, filled in 604.
Found 2902 gaps, filled in 2645.
Found 2902 gaps, filled in 2645.
Found 2902 gaps, filled in 2645.
Processed Differentials
Processed wu2s_right. Took 0:1
Found 1248 gaps, filled in 951.
Found 1248 gaps, filled in 951.
Found 1248 gaps, filled in 951.
Found 1451 gaps, filled in 1149.
Found 1451 gaps, filled in 1149.
Found 1451 gaps, filled in 1149.
Processed Differentials
Processed wu2s_center. Took 0:1
Found 2488 gaps, filled in 2346.
Found 2488 gaps, filled in 2346.
Found 2488 gaps, filled in 2346.
Found 641 gaps, filled in 513.
Found 641 gaps, filled in 513.
Found 641 gaps, filled in 513.
Processed Differentials
Processed drwn_left. Took 0:1
Found 846 

In [25]:
a=range(2,5)
for b in a:
    print(b)

2
3
4
